
# Compare number of paid merchants with the revenue we got

- В период с января по апрель 2022 года по Direct + Affiliates найти мерчантов с 0 revenue и проверить эти магазины


### Импорт библиотек

In [2]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from IPython.display import display
import plotly.express as px 
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from pandas.plotting import scatter_matrix
import math
#import random
import sys
import warnings
if not sys.warnoptions:
       warnings.simplefilter("ignore")
from collections import Counter

import requests
import re
from datetime import datetime

In [3]:
# Вывод всех строк таблицы на экран
from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_colwidth', None)


### Загрузка данных


In [5]:
# Отчет за апрель
from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

paid_april = pd.read_csv(BytesIO(r.content))
paid_april.sample()   


In [7]:
# Отчет за март
from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

paid_march = pd.read_csv(BytesIO(r.content))
paid_march.sample()   

In [8]:
# Отчет за февраль
from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

paid_febr = pd.read_csv(BytesIO(r.content))
paid_febr.sample()   

In [9]:
# Отчет за январь

from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

paid_jan = pd.read_csv(BytesIO(r.content))
paid_jan.sample()   

In [10]:
# Заливка данных по Revenue

from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

rev_raw = pd.read_csv(BytesIO(r.content))
rev_raw.sample()   

In [18]:
rev_raw.info()

In [11]:
rev_raw = rev_raw[rev_raw['application'] == 'ec']
rev_raw

In [14]:
rev_raw['application'].value_counts() 

In [ ]:
itune_raw = pd.read_csv(r'C:\Users\Marina\Downloads\revenue report_2022_05_16.csv')
itune_raw.head()

In [15]:

# Заливка данных

from io import BytesIO
import requests

spreadsheet_id = ''
file_name = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id)
r = requests.get(file_name)

revenue_raw = pd.read_csv(BytesIO(r.content))
revenue_raw.sample(10)   

### Обработка данных

In [ ]:
def str_replace(df):
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [22]:
# Заменим пробелы в названиях столбцов на _
paid_jan.columns = str_replace(paid_jan)
paid_febr.columns = str_replace(paid_febr)
paid_march.columns = str_replace(paid_march)
paid_april.columns = str_replace(paid_april)

In [19]:
paid_jan.info()

In [20]:
paid_febr.info()

In [21]:
paid_march.info()

In [22]:
paid_april.info()

In [27]:
# Заменим пробелы в названиях столбцов на _
rev_raw.columns = rev_raw.columns.str.replace(' ', '_')

In [28]:
rev = rev_raw[['store_id', 'revenue_sum,_usd', 'revenue_date:_month']].copy()

In [ ]:
itune = itune_raw[['store_id', 'revenue_sum', 'period']].copy()
itune.head()

### Используемые функции

In [30]:
# Разбивка по периодам для просмотра
def period_count(df):
    print(df['period'].value_counts())

In [31]:
# Создание таблиц по периодам
def period(df, x1):
    df1 = df[df['period'] == x1]
    return df1

In [32]:
# Вывод размера таблиц после смерживния
def month_stat(df1, df2, df3):
    df_raw = df1.append(df2, sort='False')
    
    df = df_raw.pivot_table(index='store_id', values='revenue', aggfunc = 'sum').reset_index()
    
    display(pd.isnull(df['revenue']).value_counts())
    df_all = df3.merge(df, on='store_id', how='left')
    print(df_all.shape)
    # Проверка revenue на 0
    print(pd.isnull(df_all['revenue']).value_counts())

In [33]:
# Создание таблиц с значением -2 вместо nan
def month(df1, df2):
    
    df_all = df1.merge(df2, on='store_id', how='left')
    
    df_all['total_revenue'] = df_all['total_revenue'].fillna(-2)
    
    return df_all

In [34]:
# Список мерчантов с nan после мержа с 97 отчетом помесячно
def func_nan(df):
    df1 = df[df['total_revenue'] == -2]
    sid = df1['store_id'].unique()
    return sid

In [35]:
# Список мерчантов у кого 0 в revenue после мержа
def func_zero(df):
    df1 = df[df['total_revenue'] == 0]
    sid = df1['store_id'].unique()
    return sid

In [36]:
# поиск одинаковых SID в месяцах
def n_cross(df1, df2, df3, df4, x):
    # январь по down с февралем
    common_items_1 = list((Counter(df1) & Counter(df2)).elements())
    #return common_items_1
    print(f' Одинаковые SID в январе- феврале, {x} {len(common_items_1)}') 
    # февраль по down с мартом
    common_items_2 = list((Counter(df2) & Counter(df3)).elements())
    #return common_items_2
    print(f' Одинаковые SID в феврале-марте, {x} {len(common_items_2)}')  
    # март по down с апрелем
    common_items_3 = list((Counter(df3) & Counter(df4)).elements())
    #return common_items_3
    print(f' Одинаковые SID в марте-апреле {x}  {len(common_items_3)}')  

In [38]:
# поиск одинаковых SID в 4 месяцах
def month_four(df1, df2, df3, df4, x):
    common_items_1 = list((Counter(df1) & Counter(df2) & Counter(df3) & Counter(df4)).elements())
    #return common_items_1
    print(f'Одинаковые SID по всем 4 месяцам {x} {len(common_items_1) }')



In [39]:
# Revenue по 203 отчету и 129+567
def total(df, df1, df2, lst):
    for value in lst:
        k = df[df['period'] == value]['revenue'].sum()
        a = df1[df1['period'] == value]['revenue'].sum()
        b = df2[df2['period'] == value]['revenue'].sum()
        print(f'Revenue в {value} == {k} по 203 отчету')
        print()
        print(f'Revenue в {value} == {a + b} по 567 и 129 отчету')
        print()
        print(f'Difference revenue в {value} == {k - a - b} по 203 отчету')
        print()

### Расчетная часть

In [ ]:
itune_1 = itune.pivot_table(index=['store_id', 'period'], \
                          values='revenue_sum', aggfunc = 'sum') \
.reset_index().rename(columns={'revenue_sum' : 'revenue'})

itune_1.head()

In [ ]:
rev_1 = rev.pivot_table(index=['store_id', 'revenue_date:_month'], \
                        values='revenue_sum,_usd', \
                        aggfunc = 'sum') \
.reset_index() \
.rename(columns={'revenue_sum,_usd' : 'revenue', \
                 'revenue_date:_month' : 'period'})

rev_1.head()

In [43]:
rev_1['type_revenue'] = 'internal'
itune_1['type_revenue'] = 'itunes'

In [ ]:
df_total = rev_1.append(itune_1, sort='False')
df_total.head()

In [45]:
df_total_pivot = df_total.pivot_table(index=['store_id', 'period'], \
                                      columns='type_revenue', \
                                      values='revenue', \
                                      aggfunc='sum').reset_index().fillna(0)

df_total_pivot['total'] = df_total_pivot['internal'] + df_total_pivot['itunes']

In [ ]:
df_total_pivot.columns= ['store_id', 'period', \
                         'internal_revenue', 'itunes_revenue', 'total_revenue']
df_total_pivot

In [47]:
df_jan = period(df_total_pivot, '2022-01')
df_feb = period(df_total_pivot, "2022-02")
df_mar = period(df_total_pivot,"2022-03")
df_apr = period(df_total_pivot, "2022-04")

### January 2022

In [ ]:
paid_jan['store_id'].count()

In [ ]:
all_jan = month(paid_jan, df_jan)
all_jan.head()

In [ ]:
# список кого нет в январе
jan_nan = func_nan(all_jan)
jan_nan

In [51]:
len(jan_nan)

1170

In [52]:
# Список у кого 0
jan_zero = func_zero(all_jan)
len(jan_zero)

181

### February

In [ ]:
paid_febr['store_id'].count()

In [ ]:
all_feb = month(paid_febr, df_feb)
all_feb.head()

In [ ]:
all_feb.info()

In [ ]:
# список кого нет в феврале
feb_nan = func_nan(all_feb)
feb_nan

In [56]:
# Список у кого 0
feb_zero = func_zero(all_feb)
len(feb_zero)

190

### March

In [ ]:
all_mar = month(paid_march, df_mar)
all_mar.head()

In [ ]:
# список кого нет в марте
mar_nan = func_nan(all_mar)
mar_nan

In [59]:
# Список у кого 0
mar_zero = func_zero(all_mar)
len(mar_zero)

272

### April

In [ ]:
all_apr = month(paid_april, df_apr)
all_apr.head()

In [ ]:
# список кого нет в апреле
apr_nan = func_nan(all_apr)
apr_nan

In [62]:
# Список у кого 0
apr_zero = func_zero(all_apr)
len(apr_zero)

281

### Общая часть

**Переменные:**
- jan_down, jan_sid, 
- feb_down, feb_sid,
- march_down, march_sid,
- april_down, april_sid

In [63]:
n_cross(jan_nan, feb_nan, mar_nan, apr_nan, 'у кого NAN в 97 отчете')

 Одинаковые SID в январе- феврале, у кого NAN в 97 отчете 329
 Одинаковые SID в феврале-марте, у кого NAN в 97 отчете 335
 Одинаковые SID в марте-апреле у кого NAN в 97 отчете  339


In [64]:
n_cross(jan_zero, feb_zero, mar_zero, apr_zero, 'у кого revenue = 0 в 97 отчете')

 Одинаковые SID в январе- феврале, у кого revenue = 0 в 97 отчете 164
 Одинаковые SID в феврале-марте, у кого revenue = 0 в 97 отчете 179
 Одинаковые SID в марте-апреле у кого revenue = 0 в 97 отчете  250


In [65]:
month_four(jan_nan, feb_nan, mar_nan, apr_nan, 'у кого NAN в 97 отчете')

Одинаковые SID по всем 4 месяцам у кого NAN в 97 отчете 226


In [66]:
month_four(jan_zero, feb_zero, mar_zero, apr_zero, 'у кого revenue = 0 в 97 отчете')

Одинаковые SID по всем 4 месяцам у кого revenue = 0 в 97 отчете 149


In [68]:
lst =['2022-01', '2022-02', '2022-03', '2022-04']      

In [69]:
def total(df, df1, lst):
    for value in lst:
        k = df[df['period'] == value]['revenue'].sum()
        a = df1[df1['period'] == value]['total_revenue'].sum()
        print(f'Revenue в {value} == {k} по 203 отчету')
        print()
        print(f'Revenue в {value} == {a} по 567 и 129 отчету')
        print()
        print(f'Difference revenue в {value} == {k - a} по 203 отчету')
        print()

In [17]:
total(revenue_raw, df_total_pivot, lst)

In [71]:
# поиск одинаковых SID в 4 месяцах
def month_four_1(df1, df2, df3, df4, x):
    common_items_1 = list((Counter(df1) & Counter(df2) & Counter(df3) & Counter(df4)).elements())
    return common_items_1

In [ ]:
nan_four_lst = month_four_1(jan_nan, feb_nan, mar_nan, apr_nan, 'кого нет в 97 отчете')
nan_four = pd.DataFrame({'store_id': nan_four_lst})

nan_four

In [73]:
nan_four.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   store_id  226 non-null    int64
dtypes: int64(1)
memory usage: 1.9 KB


In [77]:
df = pd.DataFrame({'statistics': ['total_paid', 'n_Nan'], \
                   'jan_22': [paid_jan['store_id'].count(), len(jan_nan)], \
                  'feb_22': [paid_febr['store_id'].count(), len(feb_nan)], \
                  'mar_22' : [paid_march['store_id'].count(), len(mar_nan)], \
                  'apr_22' : [paid_april['store_id'].count(), len(apr_nan)]})

In [79]:
jan_nan_1 = pd.DataFrame({'store_id': jan_nan})


In [80]:
feb_nan_1 = pd.DataFrame({'store_id': feb_nan})

In [81]:
mar_nan_1 = pd.DataFrame({'store_id': mar_nan})

In [82]:
apr_nan_1 = pd.DataFrame({'store_id': apr_nan})

In [142]:
# writer = pd.ExcelWriter('stat_paid_2.xlsx') 
# df.to_excel(writer, sheet_name='whole_stat', index=False)     
# jan_rev.to_excel(writer, sheet_name='jan_22', index=False)
# feb_rev.to_excel(writer, sheet_name='feb_22', index=False)
# mar_rev.to_excel(writer, sheet_name='mar_22', index=False)
# apr_rev.to_excel(writer, sheet_name='apr_22', index=False)
# nan_four_rev_1.to_excel(writer, sheet_name='four_months', index=False)
# writer.save()
# writer.close()